In [42]:
import re
import pandas as pd
import numpy as np

In [49]:
df = pd.read_csv('update_twelve_three.csv')

print(df.shape)

df.head()

(11198, 12)


,From,Message,Subject,Tags,UID,text,tag_list,first_tag,sender_name,sender_email,domain_name,isNoReply
0,<grangepayments@westernunionspeedpay.com>,"Dear AVRAHAM JACOBSOHN, This is to confirm th...",Grange Payment Confirmation,Finance,31780,<grangepayments@westernunionspeedpay.com> Gran...,['Finance'],Finance,NaN,grangepayments@westernunionspeedpay.com,westernunionspeedpay,False
1,Chase <no.reply.alerts@chase.com>,This is an Alert to help manage your account ...,Your Debit Card Transaction,Finance,31779,Chase <no.reply.alerts@chase.com> Your Debit C...,['Finance'],Finance,Chase,no.reply.alerts@chase.com,chase,True
2,Amazon Web Services <no-reply-aws@amazon.com>,Please let us know if we helped resolve your i...,Resolved 6559329691: Limit Increase: SageMaker,Productivity,31738,Amazon Web Services <no-reply-aws@amazon.com> ...,['Productivity'],Productivity,Amazon Web Services,no-reply-aws@amazon.com,amazon,True
3,Lambda Labs <noreply@github.com>,Youve been added to the Labs 18 - Tagger team ...,Bernie Durfee added you to the Lambda Labs tea...,Productivity,31693,Lambda Labs <noreply@github.com> Bernie Durfee...,['Productivity'],Productivity,Lambda Labs,noreply@github.com,github,True
4,Amazon Web Services <no-reply-aws@amazon.com>,"Hello, We haven't heard back from you regard...",Attention required on case 6559329691: Limit I...,Productivity,31684,Amazon Web Services <no-reply-aws@amazon.com> ...,['Productivity'],Productivity,Amazon Web Services,no-reply-aws@amazon.com,amazon,True


In [50]:
# Fixed the null problem for Subject and Message
# Still have the problem with sender name
df.isnull().sum()

From              0
Message           0
Subject           0
Tags              0
UID               0
text              0
tag_list          0
first_tag         0
sender_name     414
sender_email      0
domain_name       0
isNoReply         0
dtype: int64

In [51]:
# For now, replace NaN with ' '
# Hopefully this will prevent it from showing up as NaN in the future
df['sender_name'].fillna(' ', inplace=True)
df.isnull().sum()

From            0
Message         0
Subject         0
Tags            0
UID             0
text            0
tag_list        0
first_tag       0
sender_name     0
sender_email    0
domain_name     0
isNoReply       0
dtype: int64

In [52]:
# Dive in and clean messages
df['Message']

0        Dear AVRAHAM JACOBSOHN,  This is to confirm th...
1         This is an Alert to help manage your account ...
2        Please let us know if we helped resolve your i...
3        Youve been added to the Labs 18 - Tagger team ...
4         Hello,  We haven't heard back from you regard...
                               ...                        
11193    Today's highlights    Understanding Random For...
11194                                                  ...
11195    Thank you for attending AWS Machine Learning W...
11196      Hello again,     I hope you're having a nice...
11197    Today's highlights    How To Wake Up at 5 A.M....
Name: Message, Length: 11198, dtype: object

In [53]:
df.iloc[1611]['Message']

'Psst. Shipping&#x27;s FREE on orders over $49 (even the big stuff).      FR=\nEE Shipping Over $49*           [https://www.wayfair.com]    =E2=80=8C =E2=\n=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=\n=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C\n=E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=\n=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=\n=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =\n=E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=\n=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=\n=8C =E2=80=8C =E2=80=8C =E2=80=8C\n=E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=\n=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=\n=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =\n=E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=\n=8

In [48]:
df['Message'] = df['Message'].apply(lambda x: ''.join(x.split('=\r\n')))
df['Message'] = df['Message'].apply(lambda x: ''.join(x.split('=E2=80=99')))
df['Message'] = df['Message'].apply(lambda x: ' '.join(x.split('=0A')))
df['Message'] = df['Message'].apply(lambda x: ' '.join(x.split('=0D')))
df['Message'] = df['Message'].apply(lambda x: ' '.join(x.split('\r\n')))
df['Message'] = df['Message'].apply(lambda x: ' '.join(x.split('\n')))
df['Message'] = df['Message'].apply(lambda x: ' '.join(x.split('\r')))
df['Message'] = df['Message'].apply(lambda x: ' '.join(x.split('\t')))
#df['Message'] = df['Message'].apply(lambda x: ' '.join(x.split('')))
df['Message'] = df['Message'].apply(lambda x: ' '.join(x.split('=E2')))
df['Message'] = df['Message'].apply(lambda x: ' '.join(x.split('=80')))
df['Message'] = df['Message'].apply(lambda x: ' '.join(x.split('=8C')))

In [54]:
df['Message'] = df['Message'].apply(lambda x: ' '.join(x.split(r'=[A-Z0-9]{2}')))

In [57]:
pattern = '=[A-Z0-9]{2}'
match = re.findall(pattern, df.iloc[1611]['Message'])

match

['=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',
 '=8C',
 '=E2',
 '=80',


In [55]:

df.iloc[1611]['Message']

'Psst. Shipping&#x27;s FREE on orders over $49 (even the big stuff).      FR=\nEE Shipping Over $49*           [https://www.wayfair.com]    =E2=80=8C =E2=\n=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=\n=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C\n=E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=\n=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=\n=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =\n=E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=\n=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=\n=8C =E2=80=8C =E2=80=8C =E2=80=8C\n=E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=\n=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=\n=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =\n=E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=80=8C =E2=\n=8

In [12]:
df['Message'] = df['Message'].apply(text_cleaner)

TypeError: replace() argument 1 must be str, not list

### Just checking fioles for Sam

In [26]:
pd.read_csv('final_all_emails_with_data.csv')

,Delivered-To,Received,X-Google-Smtp-Source,X-Received,ARC-Seal,ARC-Message-Signature,ARC-Authentication-Results,Return-Path,Received.1,Received-SPF,...,Date,From,Reply-To,To,Message-ID,Subject,MIME-Version,Content-Type,X-Original-To,List-Unsubscribe
0,avraham.jacobsohn@gmail.com,by 2002:a4f:344a:0:0:0:0:0 with SMTP id b71csp...,APXvYqwDTQzFFTFtc7ACF3AfCyE2yLmSPNLNj9/PnmlN/B...,by 2002:a17:906:14d4:: with SMTP id y20mr28576...,['ARC-Seal'],i=1; a=rsa-sha256; c=relaxed/relaxed; d=google...,i=1; mx.google.com;\r\n dkim=pass header...,<bounces+317296-62ed-avraham.jacobsohn=gmail.c...,by 2002:a4f:344a:0:0:0:0:0 with SMTP id b71csp...,pass (google.com: domain of bounces+317296-62e...,...,"Tue, 12 Nov 2019 14:21:43 +0000 (UTC)","""Pocket"" <noreply@getpocket.com>",['Reply-To'],avraham.jacobsohn@gmail.com,<85wGrov_RdCXPsZEvxvGjA@ismtpd0059p1iad1.sendg...,3 Retirement Myths Debunked,1.0,multipart/alternative; boundary=67a8731204669b...,NaN,NaN
1,avrahamjacobsohn@gmail.com,by 2002:a4f:344a:0:0:0:0:0 with SMTP id b71csp...,APXvYqwGJjtTAPKYfPOJ/32+3mR57FbITaLDpTVGFxooYD...,by 2002:ac8:35a4:: with SMTP id k33mr32376798q...,['ARC-Seal'],i=1; a=rsa-sha256; c=relaxed/relaxed; d=google...,i=1; mx.google.com;\r\n dkim=pass header...,<201911121415517fab6af1c4394e33b98b7cdfc960p0n...,by 2002:a4f:344a:0:0:0:0:0 with SMTP id b71csp...,pass (google.com: domain of 201911121415517fab...,...,"Tue, 12 Nov 2019 14:15:51 +0000",Goodreads <no-reply@mail.goodreads.com>,['Reply-To'],avrahamjacobsohn@gmail.com,<0100016e5ff88058-4feb35e9-be25-498d-ae81-88c1...,Vote in the Semifinal Round of the Goodreads C...,1.0,"multipart/alternative; \r\n\tboundary=""----=_P...",NaN,NaN
2,avrahamjacobsohn@gmail.com,by 2002:a4f:344a:0:0:0:0:0 with SMTP id b71csp...,APXvYqw4PzdozXgbruHpRGO+wAesyC96k7QuMIH/TJ4UgD...,by 2002:a25:2443:: with SMTP id k64mr24873018y...,['ARC-Seal'],i=1; a=rsa-sha256; c=relaxed/relaxed; d=google...,i=1; mx.google.com;\r\n dkim=pass header...,<bounce-46_HTML-293582484-163655-7284872-24534...,by 2002:a4f:344a:0:0:0:0:0 with SMTP id b71csp...,pass (google.com: domain of bounce-46_html-293...,...,"Tue, 12 Nov 2019 07:43:48 -0600","""Target"" <targetnews@em.target.com>",['Reply-To'],<AVRAHAMJACOBSOHN@GMAIL.COM>,<ee744753-69fc-4d17-a0a9-cbe28ca9c239@atl1s07m...,Check out the Black Friday ad & start planning...,1.0,"multipart/alternative;\r\n\tboundary=""X8t7Kvvw...",NaN,<mailto:leave-fd8110761a3c402029-fe22167976610...
3,avrahamjacobsohn@gmail.com,by 2002:a4f:344a:0:0:0:0:0 with SMTP id b71csp...,APXvYqwqLWmBEGHnZIj3Wy1ADZwqbG5mT6spFiCowBXPxF...,by 2002:a5d:8156:: with SMTP id f22mr19816179i...,['ARC-Seal'],i=1; a=rsa-sha256; c=relaxed/relaxed; d=google...,i=1; mx.google.com;\r\n dkim=pass header...,<pm_bounces@pm-bounces.indiehackers.com>,by 2002:a4f:344a:0:0:0:0:0 with SMTP id b71csp...,pass (google.com: domain of pm_bounces@pm-boun...,...,12 Nov 2019 08:10:06 -0500,"""Indie Hackers"" <community@indiehackers.com>",['Reply-To'],avrahamjacobsohn@gmail.com,<b27a5809-f1a0-4925-8006-f0b1b7703e3c@mtasv.net>,Top Milestones: Indie Hackers is hiring!,1.0,text/html; charset=UTF-8,NaN,NaN
4,avrahamjacobsohn@gmail.com,by 2002:a4f:344a:0:0:0:0:0 with SMTP id b71csp...,APXvYqzBxQ/mePZIF9cTpZj4DDzk41oNBcYSAs+U7V0Qnz...,by 2002:a5e:db07:: with SMTP id q7mr10506519io...,['ARC-Seal'],i=1; a=rsa-sha256; c=relaxed/relaxed; d=google...,i=1; mx.google.com;\r\n dkim=pass header...,<bounce+daefca.4d1858-avrahamjacobsohn=gmail.c...,by 2002:a4f:344a:0:0:0:0:0 with SMTP id b71csp...,pass (google.com: domain of bounce+daefca.4d18...,...,"Tue, 12 Nov 2019 12:51:24 +0000",Normcore Tech <vicki@substack.com>,['Reply-To'],avrahamjacobsohn@gmail.com,<20191112125124.1.1CB6FDB5281E1EA7@mg2.substac...,Edward Snowden and our great cross-hatch,1.0,"multipart/alternative; boundary=""fe31bdbdf6cf4...",NaN,<https://vicki.substack.com/action/disable_ema...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31726,avraham.jacobsohn@gmail.com,by 10.231.15.9 wi